In [ ]:
!pip3 install -q -U kaggle_environments
!pip3 install pycodestyle==2.7.0
!pip3 install flake8==3.9.2
!mkdir agents

In [ ]:
# 0 - rock
# 1 - paper
# 2 - scissors

In [ ]:
import pandas as pd

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [ ]:
%%writefile agents/rock_agent.py

def rock_agent(observation, configuration):
    '''
        Агент всегда возвращает камень.
    '''
    return 0

In [ ]:
%%writefile agents/paper_agent.py

def paper_agent(observation, configuration):
    '''
        Агент всегда возвращает бумагу.
    '''
    return 1

In [ ]:
%%writefile agents/scissors_agent.py

def scissors_agent(observation, configuration):
    '''
        Агент всегда возвращает ножницы.
    '''
    return 2

In [ ]:
%%writefile agents/random_agent.py

import random

random.seed(0)


def random_agent(observation, configuration):
    '''
        Агент всегда возвращает случайным образом камень, ножницы или бумагу.
    '''
    return random.randint(0, 2)

In [ ]:
%%writefile agents/mode_agent.py

import random
from statistics import multimode

random.seed(0)

history = []


def mode_agent(observation, configuration):
    global history

    if observation.step > 0:
        history.append(observation.lastOpponentAction)
        mode = multimode(history)
        random_mode = random.choice(mode)
        return (random_mode + 1) % 3
    if observation.step == 0:
        return random.randint(0, 2)

In [ ]:
%%writefile agents/top10_mode_agent.py

import random
from statistics import multimode

random.seed(0)

history = []


def top10_mode_agent(observation, configuration):
    global history

    if observation.step > 0:
        history.append(observation.lastOpponentAction)
        border = min(len(history), 10)
        mode = multimode(history[-border:])
        random_mode = random.choice(mode)
        return (random_mode + 1) % 3
    if observation.step == 0:
        return random.randint(0, 2)

In [ ]:
%%writefile agents/probability_agent.py

import random
import numpy as np

random.seed(0)

history = []


def probability_agent(observation, configuration):
    global history

    if observation.step > 0:
        history.append(observation.lastOpponentAction)

        hist_array = np.array(history)
        uniq_elem, count_elem = np.unique(hist_array, return_counts=True)
        probs = count_elem / hist_array.shape[0]

        choice = random.choices(
            population=uniq_elem,
            weights=probs,
            k=1,
        )

        return (int(choice[0]) + 1) % 3
    if observation.step == 0:
        return random.randint(0, 2)

In [ ]:
%%writefile agents/random_strat_agent.py

import random
import numpy as np

random.seed(0)

history = []


def probability_agent(observation, configuration):
    global history

    if observation.step > 0:
        history.append(observation.lastOpponentAction)

        hist_array = np.array(history)
        uniq_elem, count_elem = np.unique(hist_array, return_counts=True)
        probs = count_elem / hist_array.shape[0]

        choice = random.choices(
            population=uniq_elem,
            weights=probs,
            k=1,
        )

        return (int(choice[0]) + 1) % 3
    if observation.step == 0:
        return random.randint(0, 2)

In [ ]:
%%writefile agents/random_strat_agent.py

import random

random.seed(0)

choices = [0, 1, 2]
valid = random.sample(choices, k=2)


def random_strat_agent(observation, configuration):
    global valid
    return random.choice(valid)

In [ ]:
%%writefile agents/lifehacker_agent.py

import random

random.seed(0)

self_actions = []
rewards = []


def _last_round():
    global rewards

    if len(rewards) == 1:
        if sum(rewards) == 1:
            return 'WIN'
        if sum(rewards) == 0:
            return 'DRAW'
        if sum(rewards) == -1:
            return 'LOSE'
    else:
        if sum(rewards) > sum(rewards[:-1]):
            return 'WIN'
        if sum(rewards) == sum(rewards[:-1]):
            return 'DRAW'
        if sum(rewards) < sum(rewards[:-1]):
            return 'LOSE'


def _winning_choice():
    global self_actions

    if self_actions[-1] == 0:
        return 2
    if self_actions[-1] == 2:
        return 1
    if self_actions[-1] == 1:
        return 0


def _losing_choice():
    global self_actions

    if self_actions[-1] == 0:
        return 1
    if self_actions[-1] == 2:
        return 0
    if self_actions[-1] == 1:
        return 2


def lifehacker_agent(observation, configuration):
    '''
    Если вы победили в последней игре:
        - выкинув камень, переходите к ножницам в следующей схватке
        - выкинув ножницы, переходите к бумаге в следующей схватке
        - выкинув бумагу, переходите к камню в следующей схватке

    Если вы проиграли в последней игре:
        - выкинув камень, переходите к бумаге в следующей схватке
        - выкинув ножницы, переходите к камню в следующей схватке
        - выкинув бумагу, переходите к ножницам в следующей схватке
    '''
    global self_actions
    global rewards

    choice = random.randint(0, 2)

    if observation.step > 0:
        rewards.append(observation.reward)
        if _last_round() == 'WIN':
            choice = _winning_choice()
        if _last_round() == 'LOSE':
            choice = _losing_choice()
        if _last_round() == 'DRAW':
            pass

    self_actions.append(choice)
    return choice

In [ ]:
%%writefile agents/stuborn_agent.py

import random

random.seed(0)

self_actions = []
rewards = []


def _last_round():
    global rewards

    if len(rewards) == 1:
        if sum(rewards) == 1:
            return 'WIN'
        if sum(rewards) == 0:
            return 'DRAW'
        if sum(rewards) == -1:
            return 'LOSE'
    else:
        if sum(rewards) > sum(rewards[:-1]):
            return 'WIN'
        if sum(rewards) == sum(rewards[:-1]):
            return 'DRAW'
        if sum(rewards) < sum(rewards[:-1]):
            return 'LOSE'


def stuborn_agent(observation, configuration):
    '''
    Если вы победили в последней игре:
        - выкидывает выигрышный знак с прошлой игры
    Если вы проиграли в последней игре:
        - выкидываем случайных знак из множества знаков
        без учета проигрышного знака с прошлого раунда
    Если вы последней игре ничья:
        - кидаем случайныхй из трех знаков
    '''
    global self_actions
    global rewards

    choice = random.randint(0, 2)
    if observation.step > 0:
        rewards.append(observation.reward)
        if _last_round() == 'WIN':
            choice = self_actions[-1]
        if _last_round() == 'LOSE':
            valid = [0, 1, 2]
            valid.remove(self_actions[-1])
            choice = random.choice(valid)
        if _last_round() == 'DRAW':
            pass

    self_actions.append(choice)
    return choice

In [ ]:
%%writefile agents/inversed_probability_agent.py

import random
import numpy as np

random.seed(0)

history = []


def inversed_probability_agent(observation, configuration):
    '''
        Первые 5 ходом рандомим, ведем разведку
        Затем смотрим нам самый популярный ход противника за
        последние 5 ходов - выдвигаем предположение,
        что сейчас он соскочит и пытаемся отразить самый
        непопулярный в прошлом ход
    '''
    global history

    choice = random.randint(0, 2)
    if observation.step > 5:
        history.append(observation.lastOpponentAction)

        hist_array = np.array(history[-5:])
        uniq_elem, count_elem = np.unique(hist_array, return_counts=True)
        probs = count_elem / hist_array.shape[0]
        probs = 1 - probs
        uniq_elem = uniq_elem.tolist()
        probs = probs.tolist()
        if 0 not in uniq_elem:
            uniq_elem.append(0)
            probs.append(1)
        if 1 not in uniq_elem:
            uniq_elem.append(1)
            probs.append(1)
        if 2 not in uniq_elem:
            uniq_elem.append(2)
            probs.append(1)

        choice = random.choices(
            population=uniq_elem,
            weights=probs,
            k=1,
        )

        choice = (int(choice[0]) + 1) % 3
    return choice

In [ ]:
!flake8 agents/rock_agent.py
!flake8 agents/paper_agent.py
!flake8 agents/scissors_agent.py
!flake8 agents/random_agent.py
!flake8 agents/mode_agent.py
!flake8 agents/top10_mode_agent.py
!flake8 agents/probability_agent.py
!flake8 agents/random_strat_agent.py
!flake8 agents/lifehacker_agent.py
!flake8 agents/stuborn_agent.py
!flake8 agents/inversed_probability_agent.py

In [ ]:
def match(contenders, steps=100):
    match_result = evaluate(
        "rps",
        contenders,
        configuration={"episodeSteps": steps},
        debug=True
    )
    match_result = match_result[0]
    return match_result

In [ ]:
cont_list = [
    "agents/rock_agent.py",
    "agents/scissors_agent.py",
    "agents/paper_agent.py",
    "agents/random_agent.py",
    "agents/mode_agent.py",
    "agents/top10_mode_agent.py",
    "agents/probability_agent.py",
    "agents/random_strat_agent.py",
    "agents/lifehacker_agent.py",
    "agents/stuborn_agent.py",
    "agents/inversed_probability_agent.py"
]

points = [0 for _ in cont_list]
score = [0 for _ in cont_list]

num = len(cont_list)
for i in range(num - 1):
    for j in range(i + 1, num):
        print(
            f'{cont_list[i]} - {cont_list[j]}'
        )
        cont_1, cont_2 = match(
            [cont_list[i], cont_list[j]]
        )
        points[i] += cont_1
        points[j] += cont_2

        if cont_1 == cont_2:
            print('DRAW')
            score[i] += 0
            score[j] += 0

        if cont_1 > cont_2:
            print(f'WIN {cont_list[i]}')
            score[i] += 1
            score[j] += -1

        if cont_1 < cont_2:
            print(f'WIN {cont_list[j]}')
            score[i] += -1
            score[j] += 1

In [ ]:
contender_result = pd.DataFrame({
    'contender_name': cont_list,
    'contender_point': points,
    'contender_score': score,
})

print('Score list:')
display(contender_result.sort_values(by='contender_score', ascending=False))
print('Point list:')
display(contender_result.sort_values(by='contender_point', ascending=False))

In [ ]:
!rm -rf agents